# BitFit Fine-tuning

### Step1 Importing Libraries

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

### Step2 load datasets

In [2]:
ds = Dataset.load_from_disk("../dataset/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

### step3 data process

In [4]:
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer

BloomTokenizerFast(name_or_path='D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [8]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

### step4 create model

In [9]:
# low_cpu_mem_usage 参数会加快加载
model = AutoModelForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=True)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### 看下 模型参数 细节

**有些是有 `bias` , 有些没有**

如：
```
(input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
```
```
(query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
```

In [12]:
model.parameters

<bound method Module.parameters of BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(42437, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05

In [16]:
for param in model.parameters():
    print("param: ", param,  "\nnumbers: ", param.numel(), "\n", "="*100)

param:  Parameter containing:
tensor([[-0.0099, -0.0048, -0.0111,  ..., -0.0426,  0.0099,  0.0212],
        [ 0.0048, -0.0127,  0.0138,  ..., -0.0448,  0.0003, -0.0120],
        [ 0.0065,  0.0239,  0.0050,  ..., -0.0431, -0.0067,  0.0137],
        ...,
        [ 0.0331,  0.0185,  0.0101,  ..., -0.0432, -0.0090, -0.0024],
        [-0.0132,  0.0039,  0.0054,  ..., -0.0396, -0.0120,  0.0035],
        [ 0.0044,  0.0011,  0.0230,  ..., -0.0393, -0.0087, -0.0084]],
       requires_grad=True) 
numbers:  43455488 
param:  Parameter containing:
tensor([0.4409, 0.3167, 0.4749,  ..., 0.0816, 0.2927, 0.6006],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([-0.0513,  0.0164,  0.0052,  ...,  0.2412,  0.0072, -0.0292],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([0.6621, 0.8457, 0.5884,  ..., 3.2090, 0.9180, 0.5586],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([-0.0662, -0.0798,  0.0445,  ..., 

In [17]:
sum(param.numel() for param in model.parameters())

345768960

### 显存占用

model size: 0.35B

model: 0.35G * 4 ~= 1.4G

gradient: 0.35G * 4 ~= 1.4G

optimizer: 0.35G * 4 * 2 ~= 2.8G

sum: 5.6G

### BitFit

In [18]:
for name, params in model.named_parameters():
    print("name: ", name, "\nparams: ", params, "\nnumbers", params.numel(),"="*100)

name:  transformer.word_embeddings.weight 
params:  Parameter containing:
tensor([[-0.0099, -0.0048, -0.0111,  ..., -0.0426,  0.0099,  0.0212],
        [ 0.0048, -0.0127,  0.0138,  ..., -0.0448,  0.0003, -0.0120],
        [ 0.0065,  0.0239,  0.0050,  ..., -0.0431, -0.0067,  0.0137],
        ...,
        [ 0.0331,  0.0185,  0.0101,  ..., -0.0432, -0.0090, -0.0024],
        [-0.0132,  0.0039,  0.0054,  ..., -0.0396, -0.0120,  0.0035],
        [ 0.0044,  0.0011,  0.0230,  ..., -0.0393, -0.0087, -0.0084]],
       requires_grad=True) 
numbers 43455488 ====================================================================================================
name:  transformer.word_embeddings_layernorm.weight 
params:  Parameter containing:
tensor([0.4409, 0.3167, 0.4749,  ..., 0.0816, 0.2927, 0.6006],
       requires_grad=True) 
numbers 1024 ====================================================================================================
name:  transformer.word_embeddings_layernorm.bias 
params

In [19]:
# bitfit
# 选择模型参数里面的所有bias部分

num_param = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()

num_param

272384

In [21]:
num_param / sum(param.numel() for param in model.parameters())  # 可以看到只占有原来的万分之一 。。。

0.0007877630195608073

### step5 configure train parameters

In [22]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

### step6 create trainer

In [23]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

### step7 model train

In [24]:
trainer.train()

Step,Training Loss
10,3.315200
20,3.399200
30,3.216600
40,3.123500
50,3.159400
60,3.148600
70,2.942100
80,2.942700
90,2.938200
100,2.853500


TrainOutput(global_step=3357, training_loss=2.768934656688064, metrics={'train_runtime': 1576.3134, 'train_samples_per_second': 17.038, 'train_steps_per_second': 2.13, 'total_flos': 3700748600918016.0, 'train_loss': 2.768934656688064, 'epoch': 0.9999255342914588})

In [25]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)

'Human: 考试有哪些技巧？\n\nAssistant: 首先，考生首先对考试内容及难度有足够了解，以便制定合理并有效的复习计划。其次，考生应对考试进行有效的准备，包括制定有效的复习方案。考生应根据考试的不同阶段和要求设置相应的计划，并且确保该计划的执行。考生还应提前做好准备，如准备好考试期间的学习用具，如个人电脑，学习方法，笔记，练习等。最后，考生还应及时将复习方案更新至最新的考试信息，以便考生能够从容应对考试。\n'

### step8 model inference

In [26]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [27]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 首先是参加考试之前，需要事先做好准备工作，确保自己的状态是合适的，在保证安全的前提下，避免犯一些错误，避免影响最终考试成绩。通常，参加考试之前，需要整理好个人资料，如姓名、出生时间、性别、户口所在地、兴趣等等。在预约考前注意事项上，可以要求提前提供相关方面的信息。'}]

## 关于 `torch.save`

`peft` 中没有实现关于 `bitfit` 的模型保存实现。需要自己完成（直接 `torch.save` 保存的是整个模型的参数，并不是 fine-tuned后）

方法步骤：

1. 把只训练的那部分参数(`bisa`) 取出来，然后 `torch.save`
2. 下次加载模型的时候，先用 `from_pretrained()` 加载模型，然后替换 `bias` 参数为之前保存的参数就是 fine-tuned后的模型了

In [28]:
# 微调后的 模型参数
state_dict = model.state_dict()

### 对比模型参数

第三个参数 `transformer.word_embeddings_layernorm.bias`

可以看到 fine-tuned 后的只有 含有 `bias` 的参数改变了，说明微调成功了

In [29]:
state_dict  # 这部分我们可以看到 含有bias 部分的数值都变了

OrderedDict([('transformer.word_embeddings.weight',
              tensor([[-0.0099, -0.0048, -0.0111,  ..., -0.0426,  0.0099,  0.0212],
                      [ 0.0048, -0.0127,  0.0138,  ..., -0.0448,  0.0003, -0.0120],
                      [ 0.0065,  0.0239,  0.0050,  ..., -0.0431, -0.0067,  0.0137],
                      ...,
                      [ 0.0331,  0.0185,  0.0101,  ..., -0.0432, -0.0090, -0.0024],
                      [-0.0132,  0.0039,  0.0054,  ..., -0.0396, -0.0120,  0.0035],
                      [ 0.0044,  0.0011,  0.0230,  ..., -0.0393, -0.0087, -0.0084]],
                     device='cuda:0')),
             ('transformer.word_embeddings_layernorm.weight',
              tensor([0.4409, 0.3167, 0.4749,  ..., 0.0816, 0.2927, 0.6006], device='cuda:0')),
             ('transformer.word_embeddings_layernorm.bias',
              tensor([-0.0469,  0.0138, -0.0024,  ...,  0.2356,  0.0053, -0.0286],
                     device='cuda:0')),
             ('transformer.h.0.inpu

In [30]:
# 筛选出bias参数
bias_params = {k: v for k, v in state_dict.items() if "bias" in k}

bias_params

{'transformer.word_embeddings_layernorm.bias': tensor([-0.0469,  0.0138, -0.0024,  ...,  0.2356,  0.0053, -0.0286],
        device='cuda:0'),
 'transformer.h.0.input_layernorm.bias': tensor([-0.0701, -0.0806,  0.0329,  ..., -0.1426, -0.0445,  0.0209],
        device='cuda:0'),
 'transformer.h.0.self_attention.query_key_value.bias': tensor([-0.1221, -0.0478,  0.2793,  ...,  0.0178, -0.0269,  0.0344],
        device='cuda:0'),
 'transformer.h.0.self_attention.dense.bias': tensor([-0.0296, -0.0510, -0.0341,  ...,  0.3790, -0.0060,  0.0145],
        device='cuda:0'),
 'transformer.h.0.post_attention_layernorm.bias': tensor([-0.1164, -0.2914, -0.1246,  ..., -0.1601, -0.0480, -0.0874],
        device='cuda:0'),
 'transformer.h.0.mlp.dense_h_to_4h.bias': tensor([-0.0643, -0.3338, -0.1971,  ..., -0.1335, -0.0256, -0.7752],
        device='cuda:0'),
 'transformer.h.0.mlp.dense_4h_to_h.bias': tensor([ 0.1091, -0.0226,  0.0843,  ..., -0.0394, -0.1169, -0.0177],
        device='cuda:0'),
 'transfo

In [33]:
# 只保存 bias 参数
import torch
torch.save(bias_params, r"./save/bias_params.pt")

### 加载之前保存的 `bias` 部分参数

In [40]:
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

state_dict = model.state_dict()

print("原本的模型其中一个含bias参数: " ,state_dict['transformer.word_embeddings_layernorm.bias'])

bias_params = torch.load(r"./save/bias_params.pt")

# 更新参数 （bias部分）
state_dict.update(bias_params)

model.load_state_dict(state_dict)

update_state_dict = model.state_dict()

print("\n更新后其中一个含bias: ", update_state_dict['transformer.word_embeddings_layernorm.bias'])

原本的模型其中一个含bias参数:  tensor([-0.0513,  0.0164,  0.0052,  ...,  0.2412,  0.0072, -0.0292])

更新后其中一个含bias:  tensor([-0.0469,  0.0138, -0.0024,  ...,  0.2356,  0.0053, -0.0286])
